# Import libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras import backend as K
import matplotlib.pyplot as plt
import os
import random
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.applications.resnet50 import ResNet50
K.clear_session()

# Configure GPU settings

In [ ]:
from tensorflow.python.client import device_lib
import tensorflow as tf
print(device_lib.list_local_devices())
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

# Some helpful functions for data preprocessing

In [ ]:
def get_class_names():
    out = []
    f = open("Images/labels.txt", "r")
    for label in f:
        out.append(label)
    f.close()
    return out

def get_train(size):
    f = open("Images/train.txt", "r")
    i=0
    l=0
    images = np.zeros((8126 , size[0] , size[1],3))
    labels = []
    for label in f:
        img = label.split(" ")
        path = "./"+img[0]
        im = Image.open(path)
        im = im.crop((int(img[1]),int(img[2]),int(img[3]),int(img[4]))) 
        try:
            image_data = np.array(im.resize(size))
            images[i] = image_data
            labels.append(int(img[5][:-1])-1)
            i = i+1
            l = l+1
            im.close()
        except:
            im.close()
            
    f.close()
    return images, labels

def get_test(size):
    f = open("Images/test.txt", "r")
    i=0
    l=0
    #8025
    images = np.zeros((8025 , size[0] , size[1],3))
    labels = []
    for label in f:
        img = label.split(" ")
        path = "./"+img[0]
        im = Image.open(path)
        im = im.crop((int(img[1]),int(img[2]),int(img[3]),int(img[4]))) 
        try:
            image_data = np.array(im.resize(size))
            images[i] = image_data
            labels.append(int(img[5][:-1])-1)
            i = i+1
            l = l+1
            im.close()
        except:
            im.close()
            
    f.close()
    return images, labels

# Load Train and Test data

In [ ]:
size = (224, 224)
x_train, y_train = get_train(size)
x_test, y_test = get_test(size)
class_names = get_class_names()
x_train = x_train/255
x_test = x_test/255

# Data augmentation

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(horizontal_flip=True, shear_range=0.2,zoom_range=0.2)
train_generator = datagen.flow(x_train, y_train, batch_size=24, seed=7)
datagen2 = ImageDataGenerator(horizontal_flip=True,shear_range=0.2,zoom_range=0.2)
test_generator = datagen.flow(x_test, y_test, batch_size=24)
x_train = 0
x_test = 0

# Create model - choose one of three

In [ ]:
model = keras.applications.resnet50.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None, classes=1000)
model.summary()

In [ ]:
# model = keras.applications.mobilenet.MobileNet(input_shape=None, alpha=1.0, depth_multiplier=1, dropout=1e-3, include_top=True, weights=None, input_tensor=None, pooling=None, classes=196)
# model.summary()

In [ ]:
# K.clear_session()
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(size[0],size[1],3)))
# model.add(BatchNormalization())

# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())

# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())

# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())

# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())

# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())

# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())

# model.add(Flatten())
# model.add(Dense(1024, activation='relu'))
# model.add(Dense(196, activation='softmax'))
# model.summary()

# Learning settings and training

In [ ]:
model_checkpoint = ModelCheckpoint(filepath='files/epoch-{epoch:02d}_loss-{val_loss:.4f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)

tensotboard_callback = TensorBoard(log_dir='./logs', 
            histogram_freq=0, 
            batch_size=24, 
            write_graph=True, 
            write_grads=False, 
            write_images=False, 
            embeddings_freq=0, 
            embeddings_layer_names=None, 
            embeddings_metadata=None, 
            embeddings_data=None, 
            update_freq='batch')

csv_logger = CSVLogger(filename='files/logs.csv',
                       separator=',',
                       append=True)

callbacks = [model_checkpoint,
             csv_logger,
             tensotboard_callback]

model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit_generator(train_generator, epochs = 70,steps_per_epoch = int(8126/24), callbacks = callbacks, shuffle=True, validation_data = test_generator, validation_steps = int(8025/24))